# Segmentation Notebook

In [ ]:
# from organoid import segmentation
from organoid import preprocessing
from organoid import reconstruction
import numpy as np
import tifffile

In [ ]:
path = rf"C:\Users\gros\Desktop\DATA\organoid_test"
data = tifffile.imread(rf"{path}\data.tif")
dapi = data[:, 0, :, :]
stardist_model = rf"C:\Users\gros\Desktop\CODES\Alice_Segmentation\Stardist\models\lennedist_3d_grid222_rays64"

In [ ]:
output = segmentation.predict_stardist(
    data,
    model_path=stardist_model,
    input_voxelsize=[1, 1, 1],
    normalize_input=True,
)